<div style="text-align:center">
  <h1>Thesis Code Model Estimation</h1>
  <h3>Yannick van Etten (2688877)</h3>
  <h3>Master Econometrics and Operations Research</h3>
</div>

The aim of this thesis is to determine whether election results reflect the population’s sentiments on three key issues: "Environment", "Immigration", and "Economy". Two
crucial pieces of information are used to answer this question: first, the issue score of an election, and second, the population’s stance on these issues. The data has been brought together in a different Notebook. This one will describe the final variables and implement the model.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import t
from scipy.linalg import cholesky
from scipy.stats import chi2
from IPython.display import display

<div style="text-align:center">
  <h4>Variable Set Up</h4>
  <h5>Target Independent Variables</h5>  
</div>

For all elections in this time interval, the DPES survey
includes responses to the question: "What is the most important issue facing the country?". This data is an indication of the population’s perception of the importance of
these issues. The aim is to explain the election issue scores using the percentages of people who consider a specific issue the most important. These percentages, denoted as $x_{1,t,i}$, represent the proportion of people who view issue $i$ as the most important in election $t$. Stacking the observations for all elections leads to the vector $x_{1,i}$, for all
three issues $i \in (1, 2, 3)$, representing the importance of issue $i$ across all elections.

<div style="text-align:center">
  <h5>Control Variables</h5>  
</div>

The issue importance percentage is the primary focus variable. Additionally, several control variables will be included to isolate changes in the dependent variable unrelated to the focus variable. The first control variable included is the consumer confidence indicator, which is measured monthly by Statistics Netherlands (CBS). This indicator provides insights into consumers’ confidence and perceptions regarding the developments of the Dutch economy and their own financial situation. For the consumer confidence
control variable, the indicator value of the election months will be utilized, denoted by $x_2$.

The second control variable to be utilized is the employment rate. This rate represents the unemployed labor force as a percentage of the total labor force, both employed and unemployed, and is measured by CBS. According to the  efinition provided by the International Labour Organization (ILO), individuals aged between 15 and 75 are considered unemployed if they do not have paid work, are actively  eeking employment, and are available for work. The data obtained from CBS are quarterly seasonally adjusted unemployment rates. The unemployment rate for the control variable will be based on the specific rate of the election year, denoted by $x_3$.

In the code below, the data is read and stored in a matrix called 'var_X'.

In [2]:
var_X = pd.read_csv('All_data//X_percent.csv')
var_X['Confidence'] = pd.read_csv('All_data//election_confi.csv', header=None)
var_X['Unemployment'] = pd.read_csv('All_data//election_unemp.csv', header=None)
var_X = var_X.drop(columns=['Housing', 'Employment','Healthcare','Education'])
var_X

,Environment,Immigration,Economy,Confidence,Unemployment
0,4.635762,25.662252,29.194260,-4,8.5
1,4.997620,10.756782,8.710138,33,5.1
2,9.491348,1.415836,1.678028,-9,4.1
3,11.565696,1.180173,8.418568,-29,5.4
4,2.287457,12.962257,11.056043,14,5.5
5,1.299756,8.895207,35.418359,-13,5.4
6,0.956938,5.203349,53.409091,-28,6.4
7,6.160991,20.681115,10.804954,26,5.9
8,19.233499,5.535841,4.045422,-14,4.2


<div style="text-align:center">
  <h5>Independent Variable</h5>  
</div>

To establish the issue score of an election, two sources are utilized: firstly, the national election results obtained from the Kiesraad, the authority overseeing all elections, and secondly, the score per party on these issues. This link enables the derivation of a score for each issue for every election. The party score will be a number between 0 and 10 and should indicate the party’s ability to handle this specific issue. For example, the party score for "Environment" should indicate the performance of a party in sustaining the environment and dealing with climate change.

The national election results will be denoted by $Z$, an $n \times m$ matrix where $n$ is the number of elections and $m$ is the number of political parties. Each row depicts the percentage result of all political parties. In the time interval between 1994 and 2021, 9 elections have taken place. The score per party per issue is denoted by $S_\tau^i$, an $m \times 1$ matrix, where $m$ represents the number of political parties for issue $i$. The score is determined for the intervals $\tau = 1$: 1994--2005, $\tau = 2$: 2005--2014, and $\tau = 3$: 2014--2021.

To derive an issue score for an election, the party score is multiplied by the election result percentage and summed for all parties. Note that the party score varies over time periods. To account for this variation, the matrix $I_{n}^{(t)}$ is utilized. This is an $n \times n$ identity matrix where the diagonal values are ones only for the rows representing elections in the interval where $S_\tau^i$ holds, otherwise, the diagonal value will be zero:
$$
    \textbf{y}_{i} =  I_{n}^{(1)} Z S_1^i + I_{n}^{(2)} Z S_2^i + I_{n}^{(3)} Z S_3^i
$$
where $\textbf{y}_{i}$ denotes the score for issue $i$ for all election years, in an $n \times 1$ matrix. It's important to note that the elections scores will always fall between $0$ and $10$. This is because the election percentages in $Z$ can be interpreted as weights that sum up to $100\%$, and the party scores in $S_\tau^i$ range between $0$ and $10$. For each of the three issues $i \in (1,2,3)$, a vector $\textbf{y}_{i}$ is obtained, leading to the matrix $Y = (\textbf{y}_{1}, \textbf{y}_{2}, \textbf{y}_{3})$.

In [3]:
election_environ = pd.read_csv('All_data//election_environ.csv',header = None)
election_immigration = pd.read_csv('All_data//election_immigration.csv',header = None)
election_economy = pd.read_csv('All_data//election_economy.csv',header = None)
#Y = np.column_stack([election_environ,election_immigration])
#Y = np.column_stack([Y,election_economy])
Y = pd.concat([election_environ, election_immigration, election_economy], axis=1)
Y.columns = ['Environment', 'Immigration', 'Economy']
Y

,Environment,Immigration,Economy
0,4.514790,4.703266,6.499589
1,4.643524,4.624244,6.573480
2,4.404431,5.247633,6.394663
3,4.574857,4.826890,6.418817
4,4.946841,5.139733,5.864318
5,4.769363,5.580890,5.543478
6,4.835740,5.420697,5.939929
7,5.669205,6.395269,6.052899
8,5.524533,6.320718,6.045802


<div style="text-align:center">
  <h4>Model</h4>
</div>

The aim of this thesis is to determine whether elections reflect the population’s sentiments on three key issues: "Economy", "Immigration", and "Environment". The model discussed in this research assumes a linear relationship between the percentage of the population that identifies a specific issue as the most important and the election score for that issue. Additionally, the elections score of the previous elections in included. This unrestricted model is denoted by the following equation:
$$
    \textbf{y}_{i} = \beta_{0} + \textbf{y}_{-1,i} \gamma + \textbf{x}_{1,i} * \beta_{1,i} + \textbf{x}_{2} * \beta_{2} + \textbf{x}_{3} * \beta_{3} + \epsilon_i
$$
Where $\textbf{y}_i = (y_{i,1},y_{i,2},\dots,y_{i,n})'$ represents the $n$ election score observations over time for issue $i$ and $\textbf{y}_{-1,i}$ the lagged elections scores. In this set up, an issue-dependent parameters exist. The parameter $\beta_{1,i}$ corresponding to the effect of the population issue percentage, as the relationship between the focus variable and the independent variable could vary for each issue. In this model, the control variable parameters are assumed to be consistent across different issues. To enhance the power of estimation, the three distinct linear models are merged into one by stacking the vectors of the dependent variable and structuring the $X$ matrix in a way that allows for the effect of the issue-specific focus variables to vary across all issues. This results in the following model:
$$
    \text{vec} Y = X\beta + \text{vec}\epsilon
$$
where $\beta$ a $k\times 1$ vector, $\epsilon=(\epsilon_1,\epsilon_2,\epsilon_3)'$ and
\begin{align*}
 Y &=( \textbf{y}_1, \textbf{y}_2, \textbf{y}_3)'\\
 X &=
    \begin{pmatrix}
    \textbf{1} & \textbf{y}_{-1,1} & \textbf{x}_{1,1} & \textbf{0}  & \textbf{0}  & \textbf{x}_{2} & \textbf{x}_{3} \\
    \textbf{1} & \textbf{y}_{-1,2} & \textbf{0} & \textbf{x}_{1,2}& \textbf{0}  & \textbf{x}_{2} & \textbf{x}_{3} \\
    \textbf{1} & \textbf{y}_{-1,3} & \textbf{0}  & \textbf{0} & \textbf{x}_{1,3} & \textbf{x}_{2} & \textbf{x}_{3} \\
    \end{pmatrix} 
\end{align*}
where $X$ is an $3n\times k$ matrix, $\textbf{0}$ is a vector containing $n$ zero values and $\textbf{1}$ is a vector containing $n$ one values.

In [4]:
def create_X(var_X):
    vZero = np.zeros(len(var_X))
    vX = pd.DataFrame({'Environment':np.hstack((np.array(var_X['Environment']),vZero,vZero))})
    vX['Immigration'] = np.hstack((vZero,np.array(var_X['Immigration']),vZero))
    vX['Economy'] = np.hstack((vZero,vZero,np.array(var_X['Economy'])))
    vX['Consumer Confidence'] = np.hstack((var_X['Confidence'],var_X['Confidence'],var_X['Confidence']))
    vX['Unemployment Ratio'] = np.hstack((var_X['Unemployment'],var_X['Unemployment'],var_X['Unemployment']))
    return vX

In [5]:
Y = np.array(Y)
vY_min = np.hstack(np.array(Y)[:-1,:].T)
vY_new = np.hstack(np.array(Y)[1:,:].T)
var_X_new = var_X.iloc[1:,:]
vX = create_X(var_X_new)
vX['Y-1'] = vY_min
vX = sm.add_constant(vX)
vX

,const,Environment,Immigration,Economy,Consumer Confidence,Unemployment Ratio,Y-1
0,1.0,4.997620,0.000000,0.000000,33,5.1,4.514790
1,1.0,9.491348,0.000000,0.000000,-9,4.1,4.643524
2,1.0,11.565696,0.000000,0.000000,-29,5.4,4.404431
3,1.0,2.287457,0.000000,0.000000,14,5.5,4.574857
4,1.0,1.299756,0.000000,0.000000,-13,5.4,4.946841
5,1.0,0.956938,0.000000,0.000000,-28,6.4,4.769363
6,1.0,6.160991,0.000000,0.000000,26,5.9,4.835740
7,1.0,19.233499,0.000000,0.000000,-14,4.2,5.669205
8,1.0,0.000000,10.756782,0.000000,33,5.1,4.703266
9,1.0,0.000000,1.415836,0.000000,-9,4.1,4.624244


<div style="text-align:center">
  <h4>Estimation Method</h4>
</div>

In the linear equation $\epsilon = (\epsilon_1,\epsilon_2,\epsilon_3)'$, where the disturbances $\epsilon_i$ are denoted by a $n \times 1$ vector. The assumption of $\epsilon|X \sim N(0,\sigma^2I_{3n})$ needs to be evaluated. This likely this does not hold due to interdependence of the issues, so $\hat{\beta}^{ols}$ is not efficient anymore. Therefore, we assume $\epsilon|X \sim N(0,\Omega)$. One method to deal with this situation is Generalized Least Squares (GLS), where the idea is to find $P$ such that $P'P=\Omega^{-1}$. The data can be transformed by $Py=PX\beta + Pu$. The errors of this model fulfill the Gauss Markov assumptions again, resulting in: 
$$
    \hat{\beta}^{GLS} = (X'\Omega^{-1}X)^{-1}X'\Omega^{-1}Y
$$
It is essential to define a proper structure for $\Omega$, in order to use GLS. $\Omega$ is a $3n \times 3n$ symmetric matrix. In the previous setting:
$$
    \Omega = \sigma^2 I_{3n}
$$
where the error term is assumed to be homoscedastic, independent across issues, and uncorrelated across observations and over time. However, it is reasonable to assume there are correlations between issues, and this needs to be captured in the covariance matrix. This can be accomplished by changing the structure of the covariance matrix as follows:
$$
    \Omega = \sigma^2     \begin{pmatrix}
    1 & \rho_{12}  & \rho_{13}  \\
    \rho_{12} & 1& \rho_{23}  \\
    \rho_{13}  & \rho_{23} & 1  \\
    \end{pmatrix}  
    \otimes I_{3n}
$$
where $\rho_{ij}$ denotes the correlation between issue $i$ and issue $j$ and $\sigma^2$ the variance of the residuals across all issues and elections. The Kronecker product creates a block diagonal structure so the issue correlations are repeated over time for each election. These $\rho_{ij}$ values are estimated by the correlation of the residuals of each issue, determined by the \acrshort{ols} estimation. So $\rho_{ij} = \text{corr}(e_i,e_j)$.  After obtaining these estimates, it is possible to perform the GLS estimation, where the non-zero off-diagonal elements in $\Omega$ indicate that the disturbances for different issues are correlated. GLS is repeated over and over again till the $\beta$ estimates are converged. The $\Omega$ matrix is estimated by the correlation $\rho_{ij}$ between issue $i$ and issue $j$. These are obtained using the residuals from the previous GLS step. This procedure is known as Iterated Generalized Least Squares (iGLS). 


In [6]:
def OLS(X, vY):
    X = np.array(X)
    n, o = X.shape
    beta_hat = np.linalg.inv(X.T @ X) @ X.T @ vY
    e = vY - X @ beta_hat
    s = (e.T @ e) / (n - o)
    sd = np.sqrt(np.diag(s * np.linalg.inv(X.T @ X)))
    t_stat = beta_hat / sd
    t_stat[-1] = (beta_hat[-1]-1) / sd[-1]
    return beta_hat, e, s, sd, t_stat

def iterated_GLS(X, vY, max_iter, tol):
    X_name = X.columns
    X = np.array(X)
    beta_hat, e, s, sd, t_stat = OLS(X, vY)
    nn = len(vY) // 3
    residuals_matrix = e.reshape((-1, nn)).T
    correlation_matrix = np.corrcoef(residuals_matrix, rowvar=False)
    print(np.round(correlation_matrix,3))
    for i in range(max_iter):
        residuals_matrix = e.reshape((-1, nn)).T
        correlation_matrix = np.corrcoef(residuals_matrix, rowvar=False)
        cs_block = correlation_matrix * s
        identity_n = np.eye(nn)
        Omega = np.kron(cs_block, identity_n)
        P = cholesky(Omega, lower=True)
        beta_hat_gls, e, s, sd, t_stat = OLS(np.linalg.inv(P) @ X, np.linalg.inv(P) @ vY)
        e = vY - X @ beta_hat_gls
        if np.linalg.norm(beta_hat_gls - beta_hat) < tol:
            break
        beta_hat = beta_hat_gls
    print(i)
    df = pd.DataFrame(beta_hat_gls, columns=['beta'])
    df['sd'] = sd
    df['t_stat'] = t_stat
    df.index = X_name
    df = df.round(3)
    return df, s

In [7]:
def table(df):
    df['sd'] = '(' + df['sd'].astype(str) + ')'
    df['beta'] = df['beta'].astype(str)+ ' ' + df['sd']
    df['t_stat'] = df['t_stat'].astype(str)
    df = df.drop('sd', axis=1)
    return df

def print_dfs(df, df2):
    df_la1 = table(df)
    df_la2 = table(df2)
    df_la1['beta2'] = df_la2['beta']
    df_la1['t_stat2'] = df_la2['t_stat']
    print(df_la1)

In [15]:
df, sig2_gls_unrestr = iterated_GLS(vX, vY_new, max_iter = 1, tol=1e-6)
df2, sig2_igls_unrestr = iterated_GLS(vX, vY_new, max_iter = 19, tol=1e-6)
print("Critical t-value two sided for beta:\t", t.ppf(1 - 0.05 / 2, len(vY_new) - len(df)))
print("Critical t-value one sided for gamma:\t", t.ppf(1 - 0.05, len(vY_new) - len(df)))
print_dfs(df, df2)

[[ 1.    -0.102 -0.22 ]
 [-0.102  1.    -0.215]
 [-0.22  -0.215  1.   ]]
0
[[ 1.    -0.102 -0.22 ]
 [-0.102  1.    -0.215]
 [-0.22  -0.215  1.   ]]
14
Critical t-value two sided for beta:	 2.10092204024096
Critical t-value one sided for gamma:	 1.7340636066175354
                               beta  t_stat           beta2 t_stat2
const                -0.594 (0.492)  -1.206  -0.616 (0.488)  -1.261
Environment           0.013 (0.018)   0.738   0.014 (0.018)   0.762
Immigration           0.027 (0.017)   1.627   0.027 (0.017)   1.637
Economy               0.001 (0.007)   0.078   0.001 (0.007)   0.076
Consumer Confidence   0.003 (0.003)    1.13   0.003 (0.003)   1.154
Unemployment Ratio    0.111 (0.095)  -9.319   0.115 (0.095)  -9.363


For the $\beta$ parameters the $t$-test tests $\text{H}_0 :\beta_k = 0$ and $\text{H}_1 :\beta_k \neq 0$ for all $k$ parameters. For the $\gamma$ parameter the $t$-test tests $\text{H}_0 :\gamma = 1$ and $\text{H}_1 :\gamma < 1$.

The specification of the true model is unknown. It could be the case that the lagged election is fully used, so $\gamma = 1$. The restricted model can be rewritten using: $\Delta \textbf{y}_i = \textbf{y}_i - \textbf{y}_{-1,i}$. In this setting, $\beta_{1,i}$ reflects the change in an election score related to the previous election when the percentage of people who find issue $i$ most important increases. The model is given by:
$$
    \Delta \textbf{y}_i = \beta_{0} + \textbf{x}_{1,i} * \beta_{1,i} + \textbf{x}_{2} * \beta_{2} + \textbf{x}_{3} * \beta_{3} + \epsilon_i
$$
Similar to before this results in the following model:
$$
    \text{vec}\Delta Y = X\beta + \text{vec}\epsilon
$$
where $\beta$ a $k\times 1$ vector, $\epsilon=(\epsilon_1,\epsilon_2,\epsilon_3)'$ and
\begin{align*}
\Delta Y &=(\Delta \textbf{y}_1, \Delta \textbf{y}_2,\Delta \textbf{y}_3)'\\
X &=
    \begin{pmatrix}
    \textbf{1} & \textbf{x}_{1,1} & \textbf{0}  & \textbf{0}  & \textbf{x}_{2} & \textbf{x}_{3} \\
    \textbf{1} & \textbf{0} & \textbf{x}_{1,2}& \textbf{0}  & \textbf{x}_{2} &\textbf{x}_{3} \\
    \textbf{1} & \textbf{0}  & \textbf{0} & \textbf{x}_{1,3} & \textbf{x}_{2} & \textbf{x}_{3} \\
    \end{pmatrix} 
\end{align*}

In [14]:
vY_min = np.hstack(Y[:-1,:].T)
vY_new = np.hstack(Y[1:,:].T)
vY_new = vY_new - vY_min
var_X_new = var_X.iloc[1:,:]
vX = create_X(var_X_new)
vX = sm.add_constant(vX)
vX

,const,Environment,Immigration,Economy,Consumer Confidence,Unemployment Ratio
0,1.0,4.997620,0.000000,0.000000,33,5.1
1,1.0,9.491348,0.000000,0.000000,-9,4.1
2,1.0,11.565696,0.000000,0.000000,-29,5.4
3,1.0,2.287457,0.000000,0.000000,14,5.5
4,1.0,1.299756,0.000000,0.000000,-13,5.4
5,1.0,0.956938,0.000000,0.000000,-28,6.4
6,1.0,6.160991,0.000000,0.000000,26,5.9
7,1.0,19.233499,0.000000,0.000000,-14,4.2
8,1.0,0.000000,10.756782,0.000000,33,5.1
9,1.0,0.000000,1.415836,0.000000,-9,4.1


The models are estimated using GLS and iGLS as before.

In [10]:
df, sig2_gls_beta1_1 = iterated_GLS(vX, vY_new, 1, tol=1e-6)
df2, sig2_igls_beta1_1  = iterated_GLS(vX, vY_new, 20, tol=1e-6)
print_dfs(df, df2)

[[ 1.    -0.102 -0.22 ]
 [-0.102  1.    -0.215]
 [-0.22  -0.215  1.   ]]
0
[[ 1.    -0.102 -0.22 ]
 [-0.102  1.    -0.215]
 [-0.22  -0.215  1.   ]]
14
                               beta  t_stat           beta2 t_stat2
const                -0.594 (0.492)  -1.206  -0.616 (0.488)  -1.261
Environment           0.013 (0.018)   0.738   0.014 (0.018)   0.762
Immigration           0.027 (0.017)   1.627   0.027 (0.017)   1.637
Economy               0.001 (0.007)   0.078   0.001 (0.007)   0.076
Consumer Confidence   0.003 (0.003)    1.13   0.003 (0.003)   1.154
Unemployment Ratio    0.111 (0.095)  -9.319   0.115 (0.095)  -9.363


<div style="text-align:center">
  <h4>Model Selection</h4>
</div>

The iGLS estimation procedure has been implemented. One unrestricted model, and one restricted model with $\gamma=1$ are estimated. The value of the $t$-test for $\gamma$ denoted by $\text{H}_0 :\gamma = 1$ and $\text{H}_1 :\gamma < 1$, already gives an indication of how reasonable the assumption $\gamma=1$ is. The unrestricted and restricted models will also be compared using the Wald (W), Lagrange multiplier (LM) and likelihood ratio (LR) test. These statistics are used to test the hypothesis $\text{H}_0 :\gamma = 1$. Given $\lambda = \frac{\tilde{\sigma}^2}{\hat{\sigma}^2}$, where $\tilde{\sigma}^2$ denotes the estimator of $\sigma^2$ in the unrestricted model and $\hat{\sigma}^2$ denotes the estimator of $\sigma^2$ in the restricted model. The iGLS estimates are used in this evaluation since these are equal to the maximum likelihood estimates, as stated in the paper by \cite{goldstein}. This holds under the assumption of correct model specification and convergence.

The Wald statistic is given by:
$$
    \text{W} = n(\lambda - 1)
$$
The LM statistic is given by:
$$
    \text{LM} = n(1-\lambda^{-1})
$$
The LR statistic is given by:
$$
    \text{LR} = n \text{ log } \lambda
$$

All three test statistics have a $\chi^2$-distribution with $m$ degrees of freedom, where $m$ denotes the difference in parameters between the unrestricted and restricted model, in this setting $m=1$. If the test statistics are large and the p-values are below 0.05, this would indicate that the restricted model performs worse than the unrestricted model. On the contrary, if the test statistics are small and the p-values are larger than 0.05, this would signal that the restricted model does not perform worse. In this case, the restricted model is considered adequate.

In [11]:
def model_test(sig2_unrestricted, sig2_restricted, n, npar):
    lamb = sig2_restricted / sig2_unrestricted
    LM = n* (1-1/lamb)
    LR = n * np.log(lamb)
    W = n * (lamb -1)
    p_value_LM = chi2.sf(LM, npar)
    p_value_LR = chi2.sf(LR, npar)
    p_value_W = chi2.sf(W, npar)
    return LM, LR, W, p_value_LM, p_value_LR, p_value_W

In [12]:
n = len(vY_new)
npar = 7 - 6
LM, LR, W, p_value_LM, p_value_LR, p_value_W = model_test(sig2_gls_unrestr, sig2_gls_beta1_1, n, npar)
results = pd.DataFrame({
    'Test': [ 'Lagrange Multiplier','Likelihood Ratio', 'Wald'],
    'Statistic': [LM, LR, W],
    'p-value': [p_value_LM, p_value_LR, p_value_W]
})
print('Tests based on GLS')
results.round(4)
display(results)

LM, LR, W, p_value_LM, p_value_LM, p_value_W = model_test(sig2_igls_unrestr, sig2_igls_beta1_1, n, npar)
results = pd.DataFrame({
    'Test': [ 'Lagrange Multiplier','Likelihood Ratio', 'Wald'],
    'Statistic': [LM, LR, W],
    'p-value': [p_value_LM, p_value_LR, p_value_W]
})
print('Tests based on iGLS')
results.round(4)
display(results)


Tests based on GLS


,Test,Statistic,p-value
0,Lagrange Multiplier,-0.030260,1.0
1,Likelihood Ratio,-0.030241,1.0
2,Wald,-0.030222,1.0


Tests based on iGLS


,Test,Statistic,p-value
0,Lagrange Multiplier,-0.154087,1.0
1,Likelihood Ratio,-0.153594,1.0
2,Wald,-0.153104,1.0
